In [ ]:
%matplotlib inline
%config InlineBackend.figure_format = 'svg'

In [51]:
import ROOT
import rootpy
from rootpy.io import root_open
import rootpy.plotting.root2matplotlib as rplt
from matplotlib import pyplot as plt
plt.rcParams['figure.figsize'] = [9, 6]

In [52]:
job_ids = {
    '0': 'Original_DB',
    '1': 'tip_x=0um_y=-250um',
    '2': 'tip_x=0um_y=-250um_sigma=0.5',
    '3': 'tip_x=0um_y=+100um_sigma=0.5',
    '4': 'tip_x=0um_y=-100um',
    '5': 'tip_x=0um_y=+1000um',
    '6': 'tip_x=0um_y=-1000um_sigma=0.5',
    '7': 'tip_x=0um_y=-100um_sigma=0.5',
    '9': 'tip_x=0um_y=+1000um_sigma=0.5',
    '10': 'tip_x=0um_y=-500um',
    '11': 'Nominal',
    '12': 'tip_x=0um_y=-1000um',
    '13': 'tip_x=0um_y=+100um',
    '14': 'tip_x=0um_y=+500um_sigma=0.5',
    '15': 'tip_x=0um_y=+500um',
    '17': 'tip_x=0um_y=+250um_sigma=0.5',
    '18': 'tip_x=0um_y=-500um_sigma=0.5',
    '19': 'tip_x=0um_y=+250um',
    '20': 'tip_x=0um_y=+1000um_sigma=0.5_alternate',
    '21': 'tip_x=0um_y=-1000um_sigma=0.5_alternate',
    '22': 'tip_x=0um_y=-1000um_alternate',
    '23': 'tip_x=0um_y=+1000um_alternate',
}

In [53]:
from glob import glob
import re
from collections import namedtuple
fn_pattern = 'output/ganga/(\d+)/(\d+)/output/PrCheckerInfo\.txt'
ScenarioInfo = namedtuple('ScenarioInfo', [
    'velo_ghost_rate',
    'velo_eff', 'velo_clone', 'long_eff', 'long_clone',
    'best_ghost_rate',
    'best_eff', 'best_clone',
])

In [54]:
def format_label(s):
    if s == 'Original_DB' or s == 'Nominal':
        return 'Nominal'
    elif s.startswith('tip_x=0um_y='):
        if s.endswith('_alternate'):
            s = s[:-len('_alternate')]
            alternate = ' alt'
        else:
            alternate = ''
        s = s[len('tip_x=0um_y='):]
        s = s.split('_sigma=')
        
        if len(s) == 2:
            mu, sigma = s
            sigma = ' $\\sigma$ = \\SI{{{}}}{{\\micro\\meter}}'.format(sigma)
        else:
            mu, = s
            sigma = ''
        return '$\\mu$ = \\SI{{{}}}{{\\micro\\meter}}{}{}'.format(mu[:-2], sigma, alternate)
    else:
        raise ValueError(s)

```bash
for f in */*/output/Script1_Ganga_GaudiExec.log; do
awk '/PrChecker.Velo       INFO \*\*/ {print; for(i=1; i<=196; i++) {getline; print}}' $f > ${f:h}/PrCheckerInfo.txt
done
```

In [55]:
info = {}
for fn in glob('output/ganga/*/1/output/PrCheckerInfo.txt'):
    j_id, sj_id = re.match(fn_pattern, fn).groups()
    with open(fn) as f:
        lines = [list(filter(None, l.strip().split(' '))) for l in f.readlines()]
    velo_summary = lines[0]
    velo_summary_total = int(velo_summary[4])
    velo_summary_ghosts = int(velo_summary[7])
    velo_summary_ghost_average = velo_summary[14]
    velo_velo = lines[1]
    velo_velo_reco = int(velo_velo[4])
    velo_velo_total = int(velo_velo[6])
    velo_velo_clones = int(velo_velo[10])
    velo_long = lines[2]
    velo_long_reco = int(velo_velo[4])
    velo_long_total = int(velo_velo[6])
    velo_long_clones = int(velo_velo[10])
    velo_longhighpt = lines[3]
    velo_longhighpt_reco = int(velo_velo[4])
    velo_longhighpt_total = int(velo_velo[6])
    velo_longhighpt_clones = int(velo_velo[10])
    best_summary = lines[68]
    best_summary_total = int(best_summary[4])
    best_summary_ghosts = int(best_summary[7])
    best_summary_ghost_average = best_summary[13]
    best_long = lines[69]
    best_long_reco = int(best_long[4])
    best_long_total = int(best_long[6])
    best_long_clones = int(best_long[10])
    info[job_ids[j_id]] = ScenarioInfo(
        velo_ghost_rate='{:.1%} ({}%)'.format(velo_summary_ghosts/velo_summary_total, velo_summary_ghost_average),
        velo_eff='{:.1%}'.format(velo_velo_reco/velo_velo_total),
        velo_clone='{:.1%}'.format(velo_velo_clones/(velo_velo_total+velo_velo_clones)),
        long_eff='{:.1%}'.format(velo_long_reco/velo_long_total),
        long_clone='{:.1%}'.format(velo_long_clones/(velo_long_total+velo_long_clones)),
        best_ghost_rate='{:.1%} ({}%)'.format(best_summary_ghosts/best_summary_total, best_summary_ghost_average),
        best_eff='{:.1%}'.format(best_long_reco/best_long_total),
        best_clone='{:.1%}'.format(best_long_clones/(best_long_total+best_long_clones))
    )
    print(job_ids[j_id])
    print('\n'.join([' '.join(x) for x in lines[0:5]]))
    print()
    print('\n'.join([' '.join(x) for x in lines[68:73]]))
    print()
    print()
    # print(
    #     job_ids[j_id].rjust(30),
    #     'Efficiency = {:.1%}'.format(velo_velo_reco/velo_velo_total),
    #     'Clone rate = {:.1%}'.format(velo_velo_clones/(velo_velo_total+velo_velo_clones)),
    #     'Ghost rate (average) = {:.1%} ({}%)'.format(velo_summary_ghosts/velo_summary_total, velo_summary_ghost_average),
    #     sep='\t'
    # )
    # print(best_summary)

Original_DB
PrChecker.Velo INFO **** Velo 244121 tracks including 5892 ghosts [ 2.4 %], Event average 1.9 % ****
PrChecker.Velo INFO velo : 121749 from 123419 [ 98.6 %] 4650 clones [ 3.7 %], purity: 99.78 %, hitEff: 93.78 %
PrChecker.Velo INFO long : 73870 from 74224 [ 99.5 %] 1651 clones [ 2.2 %], purity: 99.82 %, hitEff: 96.36 %
PrChecker.Velo INFO long>5GeV : 47512 from 47680 [ 99.6 %] 888 clones [ 1.8 %], purity: 99.84 %, hitEff: 96.97 %
PrChecker.Velo INFO long_strange : 3642 from 3699 [ 98.5 %] 60 clones [ 1.6 %], purity: 99.19 %, hitEff: 96.95 %

PrChecker.Best INFO **** Best 287778 tracks including 35279 ghosts [12.3 %], Event average 10.4 % ****
PrChecker.Best INFO long : 73335 from 74224 [ 98.8 %] 4059 clones [ 5.2 %], purity: 99.10 %, hitEff: 90.19 %
PrChecker.Best INFO long>5GeV : 47238 from 47680 [ 99.1 %] 2225 clones [ 4.5 %], purity: 99.16 %, hitEff: 92.38 %
PrChecker.Best INFO long_strange : 3624 from 3699 [ 98.0 %] 220 clones [ 5.7 %], purity: 98.62 %, hitEff: 87.86 %


In [56]:
if 'Original_DB' in info:
    del info['Original_DB']

In [57]:
import tabulate
def key(x):
    if x[0] in ('Original_DB', 'Nominal'):
        return 0
    elif 'alternate' in x[0]:
        return int(x[0][12:12+x[0][12:].index('um')])+10000
    else:
        return int(x[0][12:12+x[0][12:].index('um')])

table = tabulate.tabulate(
    [[print(k, format_label(k)) or k, *v] for k, v in sorted(info.items(), key=key)],
    headers=['Scenario', 'Ghost rate (average)', 'Velo efficiency', 'Velo clones', 'Long efficiency',
             'Long clones', 'Best ghost rate (average)', 'Best efficiency', 'Best clones'],
    tablefmt='latex_booktabs'
)
for k in info:
    table = table.replace(k.replace('_', '\\_'), format_label(k))
print(table)

tip_x=0um_y=-1000um $\mu$ = \SI{-1000}{\micro\meter}
tip_x=0um_y=-1000um_sigma=0.5 $\mu$ = \SI{-1000}{\micro\meter} $\sigma$ = \SI{0.5}{\micro\meter}
tip_x=0um_y=-500um_sigma=0.5 $\mu$ = \SI{-500}{\micro\meter} $\sigma$ = \SI{0.5}{\micro\meter}
tip_x=0um_y=-500um $\mu$ = \SI{-500}{\micro\meter}
tip_x=0um_y=-250um $\mu$ = \SI{-250}{\micro\meter}
tip_x=0um_y=-250um_sigma=0.5 $\mu$ = \SI{-250}{\micro\meter} $\sigma$ = \SI{0.5}{\micro\meter}
tip_x=0um_y=-100um $\mu$ = \SI{-100}{\micro\meter}
tip_x=0um_y=-100um_sigma=0.5 $\mu$ = \SI{-100}{\micro\meter} $\sigma$ = \SI{0.5}{\micro\meter}
Nominal Nominal
tip_x=0um_y=+100um $\mu$ = \SI{+100}{\micro\meter}
tip_x=0um_y=+100um_sigma=0.5 $\mu$ = \SI{+100}{\micro\meter} $\sigma$ = \SI{0.5}{\micro\meter}
tip_x=0um_y=+250um $\mu$ = \SI{+250}{\micro\meter}
tip_x=0um_y=+250um_sigma=0.5 $\mu$ = \SI{+250}{\micro\meter} $\sigma$ = \SI{0.5}{\micro\meter}
tip_x=0um_y=+500um $\mu$ = \SI{+500}{\micro\meter}
tip_x=0um_y=+500um_sigma=0.5 $\mu$ = \SI{+500}{\micro

In [50]:
with open('output/ganga/0/0/output/PrCheckerInfo.txt') as f:
    for i, l in enumerate(f.readlines()):
        l = l.strip()
        if '****' in l.split(' '):
            print('')
        print(i, *filter(None, l.split(' ')))


0 PrChecker.Velo INFO **** Velo 266518 tracks including 7273 ghosts [ 2.7 %], Event average 2.1 % ****
1 PrChecker.Velo INFO velo : 133296 from 135038 [ 98.7 %] 5080 clones [ 3.7 %], purity: 99.77 %, hitEff: 93.82 %
2 PrChecker.Velo INFO long : 81025 from 81343 [ 99.6 %] 1763 clones [ 2.1 %], purity: 99.82 %, hitEff: 96.41 %
3 PrChecker.Velo INFO long>5GeV : 52458 from 52620 [ 99.7 %] 980 clones [ 1.8 %], purity: 99.85 %, hitEff: 96.97 %
4 PrChecker.Velo INFO long_strange : 3976 from 4025 [ 98.8 %] 48 clones [ 1.2 %], purity: 99.27 %, hitEff: 97.44 %
5 PrChecker.Velo INFO long_strange>5GeV : 1901 from 1927 [ 98.7 %] 16 clones [ 0.8 %], purity: 99.13 %, hitEff: 98.16 %
6 PrChecker.Velo INFO long_fromB : 531 from 532 [ 99.8 %] 17 clones [ 3.1 %], purity: 99.74 %, hitEff: 95.29 %
7 PrChecker.Velo INFO long_fromB>5GeV : 372 from 373 [ 99.7 %] 11 clones [ 2.9 %], purity: 99.80 %, hitEff: 95.68 %
8 PrChecker.Velo INFO

9 PrChecker.Forward INFO **** Forward 120250 tracks including 44208 ghos

In [ ]:
original = root_open('output/scenarios/Original_DB/hists/0/Brunel-histos.root')
plus = root_open('output/scenarios/tip_x=0um_y=+1000um/hists/0/Brunel-histos.root')
minus = root_open('output/scenarios/tip_x=0um_y=-1000um/hists/0/Brunel-histos.root')

In [ ]:
set([x.GetName().split('_')[0] for x in original.Track.PrChecker.Forward.keys()])

In [ ]:
set([x.GetName().split('_')[-1] for x in original.Track.PrChecker.Forward.keys()])

In [ ]:
keys = [x.GetName() for x in original.Track.PrChecker.Forward.keys()]

In [ ]:
def plot(var, xlim=None):
    ho = getattr(original.Track.PrChecker.Forward, var)
    hp = getattr(plus.Track.PrChecker.Forward, var)
    hm = getattr(minus.Track.PrChecker.Forward, var)
    hp.set_line_color('r')
    hm.set_line_color('b')
    rplt.hist(ho)
    rplt.hist(hp)
    rplt.hist(hm)
    plt.title(var)
    if xlim:
        plt.xlim(xlim)

In [ ]:
keys

In [ ]:
for key in keys:
    if isinstance(getattr(original.Track.PrChecker.Forward, key), rootpy.plotting.hist._Hist2D):
        print('Skipped:', key)
    else:
        plt.figure()
        plot(key)